# Title

In [1]:
pip install --upgrade openpyxl

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install nbformat

In [3]:
pip install --upgrade nbformat plotly pandas openpyxl


In [46]:
# Load libraries

import os
import pandas as pd 
import plotly.graph_objects as go
import plotly.express as px


In [5]:
# Set the directory to the specified path

os.chdir("C:/Users/Win11/Downloads/")

# Check
current_directory = os.getcwd()
current_directory

'C:\\Users\\Win11\\Downloads'

Problem definition: Find 3-4 points that generate some recommendations as to how to optimise UA and improve game monetisation

Problem components: 
* UA can be improve through channel optimisation 
* UA can be improve through spending allocation 

Other components: 
* UA can be improven via 

Exploratory data analysis 

In [38]:
# Load the Excel file
file_path = 'Sample Data.xlsx'
data = pd.read_excel(file_path, sheet_name=0, engine='openpyxl')
del data['App']

In [39]:
# Display basic information about the dataset
print("\n--- Dataset Overview ---")
print(data.info())

# Check for missing values
print("\n--- Missing Values ---")
print(data.isnull().sum())

# Display the first few rows of the dataset
print("\n--- Sample 5 Rows ---")
data.sample(5)


--- Dataset Overview ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3461 entries, 0 to 3460
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   MONTH                     3461 non-null   datetime64[ns]
 1   PLATFORM                  3461 non-null   object        
 2   NETWORK                   3461 non-null   object        
 3   COUNTRY                   3449 non-null   object        
 4   UA_SPEND                  3461 non-null   float64       
 5   UA_IMPRESSIONS            3461 non-null   int64         
 6   UA_CLICKS                 3461 non-null   int64         
 7   INSTALLS                  3461 non-null   int64         
 8   NON_COHORTED_AD_REVENUE   3461 non-null   float64       
 9   AD_REVENUE_D1             3461 non-null   float64       
 10  AD_REVENUE_D7             3461 non-null   float64       
 11  AD_REVENUE_D30            3461 non-null   float64       

,MONTH,PLATFORM,NETWORK,COUNTRY,UA_SPEND,UA_IMPRESSIONS,UA_CLICKS,INSTALLS,NON_COHORTED_AD_REVENUE,AD_REVENUE_D1,...,AD_REVENUE_D30,AD_REVENUE_COHORTED,NON_COHORTED_IAP_REVENUE,IAP_REVENUE_D1,IAP_REVENUE_D7,IAP_REVENUE_D30,IAP_REVENUE_COHORTED,RETAINED_USERS_D1,RETAINED_USERS_D7,RETAINED_USERS_D30
3038,2024-07-01,ANDROID,ADJOE,ZM,179.7,2763,2264,593,56.1638,43.846919,...,55.408083,57.281777,0.0,0.0,0.0,0.0,0.0,153,28,2
1283,2024-08-01,IOS,IMPORTED USERS,TJ,0.0,0,0,0,0.0000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0,0,0
382,2024-07-01,ANDROID,AURA IRONSOURCE,AT,0.0,0,0,0,0.0000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0,0,0
3337,2024-06-01,IOS,ORGANIC,BD,0.0,0,0,1,0.0773,0.012892,...,0.028762,0.028762,0.0,0.0,0.0,0.0,0.0,1,0,0
1046,2024-07-01,IOS,IMPORTED USERS,KW,0.0,0,0,0,0.5555,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0,0,0


In [8]:
print(data.MONTH.min())
print(data.MONTH.max())

2024-06-01 00:00:00
2024-08-01 00:00:00


### Spending analysis 

In [20]:
# Prepare data for Sankey diagram
grouped_data = data.groupby(['PLATFORM', 'NETWORK', 'COUNTRY'])['UA_SPEND'].sum().reset_index()
grouped_data = grouped_data[grouped_data['UA_SPEND'] > 1000]
grouped_data.head()

,PLATFORM,NETWORK,COUNTRY,UA_SPEND
54,ANDROID,ADACTION,US,59921.000
71,ANDROID,ADJOE,AR,1301.635
74,ANDROID,ADJOE,AU,1242.480
94,ANDROID,ADJOE,CA,3710.700
110,ANDROID,ADJOE,DE,3937.600


In [23]:

# Extract unique values and create indices
platforms = grouped_data['PLATFORM'].unique()
networks = grouped_data['NETWORK'].unique()
countries = grouped_data['COUNTRY'].unique()

nodes = list(apps) + list(platforms) + list(networks) + list(countries)
node_indices = {node: i for i, node in enumerate(nodes)}

source, target, value = [], [], []

# Platform -> Network
for _, row in grouped_data.iterrows():
    source.append(node_indices[row['PLATFORM']])
    target.append(node_indices[row['NETWORK']])
    value.append(row['UA_SPEND'])

# Network -> Country
for _, row in grouped_data.iterrows():
    source.append(node_indices[row['NETWORK']])
    target.append(node_indices[row['COUNTRY']])
    value.append(row['UA_SPEND'])


In [27]:
# Retry creating and displaying the Sankey diagram now that the dependencies are resolved
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=nodes
    ),
    link=dict(
        source=source,
        target=target,
        value=value
    )
)])

fig.update_layout(
    title_text="Spending Flow by App, Platform, Network, and Country",
    font_size=10,
    height=800  # Adjusted height for better vertical space
)

fig.show()

- The United States stands out as the largest recipient of spending, followed by countries like RU (Russia), JP (Japan), and others.

- There is a broad distribution across many smaller markets as well, indicating a globally dispersed strategy.

- The largest allocations appear in the order of Android -> AppLovin -> United States.

- Smaller networks and countries receive relatively low allocations, indicating either niche targeting or limited presence.

- AppLovin is only Android

In [44]:

# Define a list of European countries based on ISO2 codes
european_countries = [
    'AL', 'AD', 'AT', 'BY', 'BE', 'BA', 'BG', 'HR', 'CY', 'CZ', 'DK', 'EE',
    'FI', 'FR', 'DE', 'GR', 'HU', 'IS', 'IE', 'IT', 'LV', 'LI', 'LT', 'LU',
    'MT', 'MD', 'MC', 'ME', 'NL', 'MK', 'NO', 'PL', 'PT', 'RO', 'SM', 'RS',
    'SK', 'SI', 'ES', 'SE', 'CH', 'UA', 'GB', 'VA'
]

# Generate the new column for region categorization
data['REGION'] = data['COUNTRY'].apply(
    lambda x: 'US' if x == 'US' else 'EU' if x in european_countries else 'ROW'
)

# Group the data including the new region column
grouped_data = data.groupby(['PLATFORM', 'NETWORK', 'REGION'])['UA_SPEND'].sum().reset_index()


In [36]:
# Calculate percentages
grouped_data['UA_SPEND_PERCENT'] = (
    grouped_data['UA_SPEND'] / grouped_data['UA_SPEND'].sum()
) * 100

platforms = grouped_data['PLATFORM'].unique()
networks = grouped_data['NETWORK'].unique()
countries = grouped_data['REGION'].unique()

nodes = list(networks) + list(platforms) + list(countries)
node_indices = {node: i for i, node in enumerate(nodes)}

source_percent, target_percent, value_percent = [], [], []

# Network -> Platform (percentages)
for _, row in grouped_data.iterrows():
    source_percent.append(node_indices[row['NETWORK']])
    target_percent.append(node_indices[row['PLATFORM']])
    value_percent.append(row['UA_SPEND_PERCENT'])

# Platform -> Region (percentages)
for _, row in grouped_data.iterrows():
    source_percent.append(node_indices[row['PLATFORM']])
    target_percent.append(node_indices[row['REGION']])
    value_percent.append(row['UA_SPEND_PERCENT'])

# Create the Sankey diagram with percentages and modified flow order
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=nodes
    ),
    link=dict(
        source=source_percent,
        target=target_percent,
        value=value_percent
    )
)])

fig.update_layout(
    title_text="Percentual Spending Flow by Network, Platform, and Region",
    font_size=10,
    height=1000  # Adjusted height for better vertical space
)

fig.show()

In [37]:
# I want to see how revenue changes over time and I want to see also the differences in distribution
data.MONTH.unique()

<DatetimeArray>
['2024-07-01 00:00:00', '2024-06-01 00:00:00', '2024-08-01 00:00:00']
Length: 3, dtype: datetime64[ns]

### Returns analysis

In [41]:
data['net_value'] = data['AD_REVENUE_COHORTED'] + data['IAP_REVENUE_D30']
data['net_value'].sample(5)

1241     0.049007
102      0.155600
2676    46.218124
1700     0.000000
2030     0.062801
Name: net_value, dtype: float64

In [47]:
# Prepare data for the tree map
tree_data = data.groupby(['NETWORK', 'PLATFORM', 'REGION'])[['NON_COHORTED_AD_REVENUE', 'AD_REVENUE_COHORTED']].sum().reset_index()

# Create a new column for total revenue (sum of cohorted and non-cohorted revenue)
tree_data['TOTAL_REVENUE'] = tree_data['NON_COHORTED_AD_REVENUE'] + tree_data['AD_REVENUE_COHORTED']

# Filter out rows where total revenue is zero
tree_data = tree_data[tree_data['TOTAL_REVENUE'] > 0]

# Draw the tree map
fig = px.treemap(
    tree_data,
    path=['NETWORK', 'PLATFORM', 'REGION'],  # Hierarchical levels: Network -> Platform -> Region
    values='TOTAL_REVENUE',                  # Revenue as the value
    color='TOTAL_REVENUE',                   # Use revenue to color the nodes
    color_continuous_scale='Viridis',        # Choose a color scale
    title="Revenue by Channel, Platform, and Region"
)

fig.update_layout(margin=dict(t=50, l=25, r=25, b=25))  # Adjust layout margins
fig.show()

In [52]:
pip install networkx


   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------------ --------------------------- 0.5/1.7 MB 1.4 MB/s eta 0:00:01
   ------------------ --------------------- 0.8/1.7 MB 1.2 MB/s eta 0:00:01
   ------------------------ --------------- 1.0/1.7 MB 1.5 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 1.7 MB/s eta 0:00:00


In [55]:
pip install pygraphviz

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build pygraphviz
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Building wheel for pygraphviz (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [48 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build\lib.win-amd64-cpython-313\pygraphviz
      copying pygraphviz\agraph.py -> build\lib.win-amd64-cpython-313\pygraphviz
      copying pygraphviz\graphviz.py -> build\lib.win-amd64-cpython-313\pygraphviz
      copying pygraphviz\scraper.py -> build\lib.win-amd64-cpython-313\pygraphviz
      copying pygraphviz\testing.py -> build\lib.win-amd64-cpython-313\pygraphviz
      copying pygraphviz\__init__.py -> build\lib.win-amd64-cpython-313\pygraphviz
      creating build\lib.win-amd64-cpython-313\pygraphviz\tests
      copying pygraphviz\tests\test_attribute_defaults.py -> build\lib.win-amd64-cpython-313\pygraphviz\tests
      copying pygraphviz\tests\test_clear.py -> build\lib.win-amd64-cpython-313\pygraphviz\tests
      copying pygraphviz\test

In [56]:
# Redraw the graph to go from top to bottom (hierarchical layout)
import matplotlib.pyplot as plt
import networkx as nx

# Create a directed graph
G = nx.DiGraph()

plt.figure(figsize=(16, 12))

# Use a hierarchical layout for better clarity
pos = nx.nx_agraph.graphviz_layout(G, prog='dot')  # 'dot' creates a top-to-bottom hierarchy

nx.draw(
    G,
    pos,
    with_labels=True,
    node_size=3000,
    node_color="lightblue",
    font_size=10,
    font_weight="bold",
    arrowsize=10,
)
plt.title("Top-to-Bottom Hierarchical Revenue Tree Visualization", fontsize=16)
plt.show()


ImportError: requires pygraphviz http://pygraphviz.github.io/

<Figure size 1600x1200 with 0 Axes>

In [84]:
# Adjusting the provided code to include spending in addition to revenue

# Step 1: Calculate revenues and spending at each level
grouped_data = data.groupby(['NETWORK', 'PLATFORM', 'REGION'])[
    ['NON_COHORTED_AD_REVENUE', 'AD_REVENUE_COHORTED', 'UA_SPEND']
].sum().reset_index()

# Calculate total revenue (sum of cohorted and non-cohorted revenue)
grouped_data['TOTAL_REVENUE'] = grouped_data['NON_COHORTED_AD_REVENUE'] + grouped_data['AD_REVENUE_COHORTED']

# Step 2: Summarize revenues and spending for each hierarchy level
# Revenue and spending for Network level
network_revenue_spending = grouped_data.groupby('NETWORK')[
    ['TOTAL_REVENUE', 'UA_SPEND']
].sum().reset_index().rename(columns={
    'TOTAL_REVENUE': 'Network_Alone_Revenue',
    'UA_SPEND': 'Network_Alone_Spending'
})

# Revenue and spending for Platform-Network level
platform_network_revenue_spending = grouped_data.groupby(['NETWORK', 'PLATFORM'])[
    ['TOTAL_REVENUE', 'UA_SPEND']
].sum().reset_index().rename(columns={
    'TOTAL_REVENUE': 'Platform_Network_Revenue',
    'UA_SPEND': 'Platform_Network_Spending'
})

# Revenue and spending for Platform-Network-Region level
platform_network_region_revenue_spending = grouped_data.groupby(['NETWORK', 'PLATFORM', 'REGION'])[
    ['TOTAL_REVENUE', 'UA_SPEND']
].sum().reset_index().rename(columns={
    'TOTAL_REVENUE': 'Platform_Network_Region_Revenue',
    'UA_SPEND': 'Platform_Network_Region_Spending'
})

# Merge the results into a single dataset
merged_data = platform_network_region_revenue_spending.merge(
    platform_network_revenue_spending, on=['NETWORK', 'PLATFORM'], how='left'
).merge(
    network_revenue_spending, on='NETWORK', how='left'
)

# Adding ROAS (Return on Ad Spend) to the merged dataset
merged_data = merged_data[merged_data['Platform_Network_Region_Spending'] > 0]

# Calculate ROAS at each level
merged_data['Network_Alone_ROAS'] = merged_data['Network_Alone_Revenue'] / merged_data['Network_Alone_Spending']
merged_data['Platform_Network_ROAS'] = merged_data['Platform_Network_Revenue'] / merged_data['Platform_Network_Spending']
merged_data['Platform_Network_Region_ROAS'] = merged_data['Platform_Network_Region_Revenue'] / merged_data['Platform_Network_Region_Spending']




merged_data

,NETWORK,PLATFORM,REGION,Platform_Network_Region_Revenue,Platform_Network_Region_Spending,Platform_Network_Revenue,Platform_Network_Spending,Network_Alone_Revenue,Network_Alone_Spending,Network_Alone_ROAS,Platform_Network_ROAS,Platform_Network_Region_ROAS
2,ADACTION,ANDROID,US,51696.391911,59921.0000,51731.308271,59921.0000,79899.994807,88913.8000,0.898623,0.863325,0.862742
3,ADACTION,IOS,EU,4734.882872,6314.8000,28168.686536,28992.8000,79899.994807,88913.8000,0.898623,0.971575,0.749807
4,ADACTION,IOS,ROW,3159.051214,4074.0000,28168.686536,28992.8000,79899.994807,88913.8000,0.898623,0.971575,0.775418
5,ADACTION,IOS,US,20274.752450,18604.0000,28168.686536,28992.8000,79899.994807,88913.8000,0.898623,0.971575,1.089806
6,ADJOE,ANDROID,EU,48728.182702,51304.6060,113159.357721,114724.6750,119595.585606,121356.9150,0.985486,0.986356,0.949782
7,ADJOE,ANDROID,ROW,37510.727301,38016.8350,113159.357721,114724.6750,119595.585606,121356.9150,0.985486,0.986356,0.986687
8,ADJOE,ANDROID,US,26920.447718,25403.2340,113159.357721,114724.6750,119595.585606,121356.9150,0.985486,0.986356,1.059725
11,ADJOE,IOS,US,6434.073675,6632.2400,6436.227885,6632.2400,119595.585606,121356.9150,0.985486,0.970446,0.970121
14,APPLOVIN,ANDROID,EU,27771.308320,22045.4003,98956.287148,92719.6938,99291.872248,92719.6938,1.070882,1.067263,1.259733
15,APPLOVIN,ANDROID,ROW,38012.817591,37295.7101,98956.287148,92719.6938,99291.872248,92719.6938,1.070882,1.067263,1.019228


In [81]:
#  Prepare the nodes dynamically from the data
nodes = ["Total Spending"]  # Start with the top layer
network_nodes = merged_data['NETWORK'].unique().tolist()
nodes.extend(network_nodes)

# Map the network index
network_indices = {network: idx + 1 for idx, network in enumerate(network_nodes)}

platform_nodes = merged_data[['NETWORK', 'PLATFORM']].drop_duplicates()
platform_map = {}

for _, row in platform_nodes.iterrows():
    key = f"{row['PLATFORM']} ({row['NETWORK']})"
    nodes.append(key)
    platform_map[(row['PLATFORM'], row['NETWORK'])] = len(nodes) - 1

region_map = {}
region_nodes = merged_data[['NETWORK', 'PLATFORM', 'REGION']].drop_duplicates()
for _, row in region_nodes.iterrows():
    key = f"{row['REGION']} ({row['PLATFORM']}-{row['NETWORK']})"
    nodes.append(key)
    region_map[(row['REGION'], row['PLATFORM'], row['NETWORK'])] = len(nodes) - 1

# Prepare the links
sources = []
targets = []
values = []

# Total spending to networks
for network in network_nodes:
    sources.append(0)  # Total Spending node index
    targets.append(network_indices[network])
    values.append(merged_data[merged_data['NETWORK'] == network]['Network_Alone_Revenue'].iloc[0])

# Networks to platforms
for (platform, network), idx in platform_map.items():
    sources.append(network_indices[network])
    targets.append(idx)
    values.append(merged_data[(merged_data['NETWORK'] == network) & (merged_data['PLATFORM'] == platform)]['Platform_Network_Revenue'].iloc[0])

# Platforms to regions
for (region, platform, network), idx in region_map.items():
    sources.append(platform_map[(platform, network)])
    targets.append(idx)
    values.append(merged_data[(merged_data['NETWORK'] == network) & (merged_data['PLATFORM'] == platform) & (merged_data['REGION'] == region)]['Platform_Network_Region_Revenue'].iloc[0])

# Create the organizational-style chart
fig = go.Figure(go.Sankey(
    node=dict(
        pad=20,
        thickness=30,
        line=dict(color="black", width=0.5),
        label=nodes,
        color="lightblue"
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,
        color="rgba(0,100,200,0.4)"  # Transparent link color for consulting style
    )
))

fig.update_layout(
    title_text="Spending Allocation by Layer (Consulting Style)",
    font_size=14,
    title_font_size=18,
    height=700,
    margin=dict(t=50, l=50, r=50, b=50)
)

fig.show()

### We will add the dynamic view

In [116]:
# Adjusting the breakdown to include "Month-Alone Spending" and hierarchical levels

# Step 1: Calculate revenues and spending at each level, including MONTH
grouped_data = data.groupby(['MONTH', 'NETWORK', 'PLATFORM', 'REGION'])[
    ['NON_COHORTED_AD_REVENUE', 'AD_REVENUE_COHORTED', 'UA_SPEND']
].sum().reset_index()

# Calculate total revenue (sum of cohorted and non-cohorted revenue)
grouped_data['TOTAL_REVENUE'] = grouped_data['NON_COHORTED_AD_REVENUE'] + grouped_data['AD_REVENUE_COHORTED']

# Step 2: Summarize revenues and spending for each hierarchy level
# Revenue and spending for Month level alone
month_revenue_spending = grouped_data.groupby(['MONTH'])[
    ['TOTAL_REVENUE', 'UA_SPEND']
].sum().reset_index().rename(columns={
    'TOTAL_REVENUE': 'Month_Alone_Revenue',
    'UA_SPEND': 'Month_Alone_Spending'
})

# Revenue and spending for Network-Month level
network_month_revenue_spending = grouped_data.groupby(['MONTH', 'NETWORK'])[
    ['TOTAL_REVENUE', 'UA_SPEND']
].sum().reset_index().rename(columns={
    'TOTAL_REVENUE': 'Network_Month_Revenue',
    'UA_SPEND': 'Network_Month_Spending'
})

# Revenue and spending for Platform-Network-Month level
platform_network_month_revenue_spending = grouped_data.groupby(['MONTH', 'NETWORK', 'PLATFORM'])[
    ['TOTAL_REVENUE', 'UA_SPEND']
].sum().reset_index().rename(columns={
    'TOTAL_REVENUE': 'Platform_Network_Month_Revenue',
    'UA_SPEND': 'Platform_Network_Month_Spending'
})

# Revenue and spending for Platform-Network-Region-Month level
platform_network_region_month_revenue_spending = grouped_data.groupby(['MONTH', 'NETWORK', 'PLATFORM', 'REGION'])[
    ['TOTAL_REVENUE', 'UA_SPEND']
].sum().reset_index().rename(columns={
    'TOTAL_REVENUE': 'Platform_Network_Region_Month_Revenue',
    'UA_SPEND': 'Platform_Network_Region_Month_Spending'
})

# Merge the results into a single dataset
merged_data = platform_network_region_month_revenue_spending.merge(
    platform_network_month_revenue_spending, on=['MONTH', 'NETWORK', 'PLATFORM'], how='left'
).merge(
    network_month_revenue_spending, on=['MONTH', 'NETWORK'], how='left'
).merge(
    month_revenue_spending, on=['MONTH'], how='left'
)

# Adding ROAS (Return on Ad Spend) to the merged dataset
merged_data = merged_data[merged_data['Platform_Network_Region_Month_Spending'] > 0]

# Calculate ROAS at each level
merged_data['Month_Alone_ROAS'] = merged_data['Month_Alone_Revenue'] / merged_data['Month_Alone_Spending']
merged_data['Network_Month_ROAS'] = merged_data['Network_Month_Revenue'] / merged_data['Network_Month_Spending']
merged_data['Platform_Network_Month_ROAS'] = merged_data['Platform_Network_Month_Revenue'] / merged_data['Platform_Network_Month_Spending']
merged_data['Platform_Network_Region_Month_ROAS'] = merged_data['Platform_Network_Region_Month_Revenue'] / merged_data['Platform_Network_Region_Month_Spending']

# Display the resulting dataset
merged_data.sample(5)

,MONTH,NETWORK,PLATFORM,REGION,Platform_Network_Region_Month_Revenue,Platform_Network_Region_Month_Spending,Platform_Network_Month_Revenue,Platform_Network_Month_Spending,Network_Month_Revenue,Network_Month_Spending,Month_Alone_Revenue,Month_Alone_Spending,Month_Alone_ROAS,Network_Month_ROAS,Platform_Network_Month_ROAS,Platform_Network_Region_Month_ROAS
164,2024-08-01,ADJOE,IOS,US,2568.915949,2531.54,2569.926349,2531.540,31525.845680,31740.384,74254.030324,57530.8620,1.290682,0.993241,1.015163,1.014764
157,2024-08-01,ADACTION,IOS,ROW,319.965105,612.00,6366.909992,6791.200,21113.706196,20979.200,74254.030324,57530.8620,1.290682,1.006411,0.937524,0.522819
145,2024-07-01,TYRADS,IOS,ROW,213.915924,218.85,2902.961460,3606.790,10812.313498,13511.944,136955.607512,129055.9154,1.061211,0.800204,0.804860,0.977455
3,2024-06-01,ADACTION,IOS,EU,2484.285060,2882.40,14404.209756,12469.600,31686.339878,34834.600,171447.414933,166208.6783,1.031519,0.909623,1.155146,0.861881
6,2024-06-01,ADJOE,ANDROID,EU,17613.393127,17989.84,42773.722127,42022.745,43882.976758,43072.745,171447.414933,166208.6783,1.031519,1.018811,1.017871,0.979074


In [122]:
import plotly.graph_objects as go

# Prepare the nodes dynamically from the data with "Month" as the first node
nodes = []  # Start with an empty list of nodes
month_nodes = merged_data['MONTH'].unique().tolist()
nodes.extend(month_nodes)

# Map the month index
month_indices = {month: idx for idx, month in enumerate(month_nodes)}

network_nodes = merged_data[['MONTH', 'NETWORK']].drop_duplicates()
network_map = {}

for _, row in network_nodes.iterrows():
    key = f"{row['NETWORK']}"
    nodes.append(key)
    network_map[(row['MONTH'], row['NETWORK'])] = len(nodes) - 1

platform_nodes = merged_data[['MONTH', 'NETWORK', 'PLATFORM']].drop_duplicates()
platform_map = {}

for _, row in platform_nodes.iterrows():
    key = f"{row['PLATFORM']}"
    nodes.append(key)
    platform_map[(row['MONTH'], row['NETWORK'], row['PLATFORM'])] = len(nodes) - 1

region_map = {}
region_nodes = merged_data[['MONTH', 'NETWORK', 'PLATFORM', 'REGION']].drop_duplicates()
for _, row in region_nodes.iterrows():
    key = f"{row['REGION']})"
    nodes.append(key)
    region_map[(row['MONTH'], row['NETWORK'], row['PLATFORM'], row['REGION'])] = len(nodes) - 1

# Prepare the links
sources = []
targets = []
values = []

# Month to networks
for (month, network), idx in network_map.items():
    sources.append(month_indices[month])  # Month node index
    targets.append(idx)
    values.append(merged_data[(merged_data['MONTH'] == month) & (merged_data['NETWORK'] == network)]['Network_Month_Revenue'].iloc[0])

# Networks to platforms
for (month, network, platform), idx in platform_map.items():
    sources.append(network_map[(month, network)])
    targets.append(idx)
    values.append(merged_data[(merged_data['MONTH'] == month) & (merged_data['NETWORK'] == network) & (merged_data['PLATFORM'] == platform)]['Platform_Network_Month_Revenue'].iloc[0])

# Platforms to regions
for (month, network, platform, region), idx in region_map.items():
    sources.append(platform_map[(month, network, platform)])
    targets.append(idx)
    values.append(merged_data[(merged_data['MONTH'] == month) & (merged_data['NETWORK'] == network) & (merged_data['PLATFORM'] == platform) & (merged_data['REGION'] == region)]['Platform_Network_Region_Month_Revenue'].iloc[0])

# Create the organizational-style chart with month as the first node
fig = go.Figure(go.Sankey(
    node=dict(
        pad=20,
        thickness=30,
        line=dict(color="black", width=0.5),
        label=nodes,
        color="lightblue"
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,
        color="rgba(0,100,200,0.4)"  # Transparent link color for consulting style
    )
))

fig.update_layout(
    title_text="Spending Allocation by Month (Consulting Style)",
    font_size=14,
    title_font_size=18,
    height=900,
    margin=dict(t=50, l=50, r=50, b=50)
)

fig.show()


In [115]:
merged_data.sort_values(by = 'Month_Alone_Spending').sample(5)

KeyError: 'Month_Alone_Spending'

### Other views

In [109]:
# Set a maximum ROAS threshold for the thickest color
max_roas_limit = 1.52  # Example: Cap ROAS at 5

# Prepare link colors based on capped ROAS values
roas_colors = []

# Total spending to networks
for network in network_nodes:
    roas = merged_data[merged_data['NETWORK'] == network]['Network_Alone_ROAS'].iloc[0]
    capped_roas = min(roas, max_roas_limit)  # Cap ROAS at the defined limit
    roas_colors.append(capped_roas)

# Networks to platforms
for (platform, network), idx in platform_map.items():
    roas = merged_data[(merged_data['NETWORK'] == network) & (merged_data['PLATFORM'] == platform)]['Platform_Network_ROAS'].iloc[0]
    capped_roas = min(roas, max_roas_limit)  # Cap ROAS at the defined limit
    roas_colors.append(capped_roas)

# Platforms to regions
for (region, platform, network), idx in region_map.items():
    roas = merged_data[(merged_data['NETWORK'] == network) & (merged_data['PLATFORM'] == platform) & (merged_data['REGION'] == region)]['Platform_Network_Region_ROAS'].iloc[0]
    capped_roas = min(roas, max_roas_limit)  # Cap ROAS at the defined limit
    roas_colors.append(capped_roas)

# Normalize ROAS values for color intensity
roas_colors = [
    f"rgba(0, 100, 200, {-0.85 + 1.85 * (min(roas, max_roas_limit) / max_roas_limit)})"
    for roas in roas_colors
]

# Create the organizational-style chart with ROAS in labels and capped color intensity
fig = go.Figure(go.Sankey(
    node=dict(
        pad=20,
        thickness=30,
        line=dict(color="black", width=0.5),
        label=node_labels,
        color="lightblue"
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,
        color=roas_colors  # Color links based on capped ROAS
    )
))

fig.update_layout(
    title_text="ROAS Allocation by Layer with Dynamically Capped Color Intensity",
    font_size=14,
    title_font_size=18,
    height=700,
    margin=dict(t=50, l=50, r=50, b=50)
)

fig.show()


### Pattern recognition

In [60]:
import plotly.graph_objects as go

# Data: Example structure from the grouped dataset
nodes = [
    "Total Spending",  # Top layer
    "ADACTION", "ADJOE", "APPLOVIN",  # Networks
    "ANDROID (ADACTION)", "IOS (ADACTION)",  # Platforms under ADACTION
    "EU (ANDROID-ADACTION)", "ROW (ANDROID-ADACTION)", "US (ANDROID-ADACTION)",  # Regions under ANDROID-ADACTION
    "EU (IOS-ADACTION)", "ROW (IOS-ADACTION)", "US (IOS-ADACTION)"  # Regions under IOS-ADACTION
]

# Links: Define source and target relationships with values
links = dict(
    source=[
        0, 0, 0,  # Total Spending to Networks
        1, 1,  # ADACTION to Platforms
        3, 3, 3,  # ANDROID (ADACTION) to Regions
        4, 4, 4   # IOS (ADACTION) to Regions
    ],
    target=[
        1, 2, 3,  # Networks
        3, 4,  # Platforms under ADACTION
        5, 6, 7,  # Regions under ANDROID (ADACTION)
        8, 9, 10  # Regions under IOS (ADACTION)
    ],
    value=[
        79899.99, 50000, 60000,  # Spending from Total Spending to Networks
        51731.31, 28168.69,  # Spending from ADACTION to Platforms
        11.71, 23.21, 51696.39,  # Spending from ANDROID (ADACTION) to Regions
        4734.88, 3159.05, 20274.75  # Spending from IOS (ADACTION) to Regions
    ]
)

# Create the organizational-style chart
fig = go.Figure(go.Sankey(
    node=dict(
        pad=20,
        thickness=30,
        line=dict(color="black", width=0.5),
        label=nodes,
        color="lightblue"
    ),
    link=dict(
        source=links["source"],
        target=links["target"],
        value=links["value"],
        color="rgba(0,100,200,0.4)"  # Transparent link color for consulting style
    )
))

fig.update_layout(
    title_text="Spending Allocation by Layer (Consulting Style)",
    font_size=14,
    title_font_size=18,
    height=700,
    margin=dict(t=50, l=50, r=50, b=50)
)

fig.show()


In [74]:
# network_nodes = merged_data['NETWORK'].unique().tolist()
merged_data

,NETWORK,PLATFORM,REGION,Platform_Network_Region_Revenue,Platform_Network_Revenue,Network_Alone_Revenue
0,ADACTION,ANDROID,EU,11.707089,51731.308271,79899.994807
1,ADACTION,ANDROID,ROW,23.209271,51731.308271,79899.994807
2,ADACTION,ANDROID,US,51696.391911,51731.308271,79899.994807
3,ADACTION,IOS,EU,4734.882872,28168.686536,79899.994807
4,ADACTION,IOS,ROW,3159.051214,28168.686536,79899.994807
...,...,...,...,...,...,...
75,UNITY ADS,ANDROID,ROW,4.047200,25.693000,133.167900
76,UNITY ADS,ANDROID,US,2.808400,25.693000,133.167900
77,UNITY ADS,IOS,EU,15.921200,107.474900,133.167900
78,UNITY ADS,IOS,ROW,41.778700,107.474900,133.167900


In [79]:
#  Prepare the nodes dynamically from the data
nodes = ["Total Spending"]  # Start with the top layer
network_nodes = data['NETWORK'].unique().tolist()
nodes.extend(network_nodes)

# Map the network index
network_indices = {network: idx + 1 for idx, network in enumerate(network_nodes)}

platform_nodes = merged_data[['NETWORK', 'PLATFORM']].drop_duplicates()
platform_map = {}

for _, row in platform_nodes.iterrows():
    key = f"{row['PLATFORM']} ({row['NETWORK']})"
    nodes.append(key)
    platform_map[(row['PLATFORM'], row['NETWORK'])] = len(nodes) - 1

region_map = {}
region_nodes = merged_data[['NETWORK', 'PLATFORM', 'REGION']].drop_duplicates()
for _, row in region_nodes.iterrows():
    key = f"{row['REGION']} ({row['PLATFORM']}-{row['NETWORK']})"
    nodes.append(key)
    region_map[(row['REGION'], row['PLATFORM'], row['NETWORK'])] = len(nodes) - 1

# Prepare the links
sources = []
targets = []
values = []

# Total spending to networks
for network in network_nodes:
    sources.append(0)  # Total Spending node index
    targets.append(network_indices[network])
    values.append(merged_data[merged_data['NETWORK'] == network]['Network_Alone_Revenue'].iloc[0])

# Networks to platforms
for (platform, network), idx in platform_map.items():
    sources.append(network_indices[network])
    targets.append(idx)
    values.append(merged_data[(merged_data['NETWORK'] == network) & (merged_data['PLATFORM'] == platform)]['Platform_Network_Revenue'].iloc[0])

# Platforms to regions
for (region, platform, network), idx in region_map.items():
    sources.append(platform_map[(platform, network)])
    targets.append(idx)
    values.append(merged_data[(merged_data['NETWORK'] == network) & (merged_data['PLATFORM'] == platform) & (merged_data['REGION'] == region)]['Platform_Network_Region_Revenue'].iloc[0])

# Create the organizational-style chart
fig = go.Figure(go.Sankey(
    node=dict(
        pad=20,
        thickness=30,
        line=dict(color="black", width=0.5),
        label=nodes,
        color="lightblue"
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,
        color="rgba(0,100,200,0.4)"  # Transparent link color for consulting style
    )
))

fig.update_layout(
    title_text="Spending Allocation by Layer (Consulting Style)",
    font_size=14,
    title_font_size=18,
    height=700,
    margin=dict(t=50, l=50, r=50, b=50)
)

fig.show()

IndexError: single positional indexer is out-of-bounds

In [67]:
import networkx as nx
import matplotlib.pyplot as plt

# Example grouped dataset
merged_data = [
    {'NETWORK': 'Network1', 'PLATFORM': 'Platform1', 'REGION': 'Region1', 'Network_Alone_Revenue': 1000, 'Platform_Network_Revenue': 700, 'Platform_Network_Region_Revenue': 500},
    {'NETWORK': 'Network1', 'PLATFORM': 'Platform1', 'REGION': 'Region2', 'Network_Alone_Revenue': 1000, 'Platform_Network_Revenue': 700, 'Platform_Network_Region_Revenue': 200},
    {'NETWORK': 'Network1', 'PLATFORM': 'Platform2', 'REGION': 'Region1', 'Network_Alone_Revenue': 1000, 'Platform_Network_Revenue': 300, 'Platform_Network_Region_Revenue': 300},
    {'NETWORK': 'Network2', 'PLATFORM': 'Platform1', 'REGION': 'Region1', 'Network_Alone_Revenue': 500, 'Platform_Network_Revenue': 500, 'Platform_Network_Region_Revenue': 500},
]

# Create a directed graph
G = nx.DiGraph()

# Add nodes and edges for the hierarchy
for row in merged_data:
    network = row['NETWORK']
    platform = row['PLATFORM']
    region = row['REGION']
    network_revenue = f"Total: ${row['Network_Alone_Revenue']:.2f}"
    platform_revenue = f"${row['Platform_Network_Revenue']:.2f}"
    region_revenue = f"${row['Platform_Network_Region_Revenue']:.2f}"

    # Add nodes
    G.add_node(network, label=f"{network}\n{network_revenue}")
    G.add_node(f"{network}-{platform}", label=f"{platform}\n{platform_revenue}")
    G.add_node(f"{network}-{platform}-{region}", label=f"{region}\n{region_revenue}")

    # Add edges
    G.add_edge(network, f"{network}-{platform}")
    G.add_edge(f"{network}-{platform}", f"{network}-{platform}-{region}")

# Define a hierarchical layout (top-to-bottom)
pos = nx.multipartite_layout(G, subset_key=lambda n: len(n.split('-')))

# Plot the graph
plt.figure(figsize=(14, 10))
nx.draw(
    G,
    pos,
    with_labels=True,
    labels=nx.get_node_attributes(G, 'label'),
    node_size=3000,
    node_color="lightblue",
    font_size=10,
    font_weight="bold",
    arrowsize=15
)

# Add a title
plt.title("Spending Allocation Hierarchy (Network → Platform → Region)", fontsize=16)
plt.show()


NetworkXError: all nodes need a subset_key attribute: <function <lambda> at 0x0000016F3EEFE980>

# Development of the strategy

###